In [5]:
import tensorflow as tf
from tensorflow import keras
import gym
import pathlib
import numpy as np
import IPython.display as display
import sys
from matplotlib import pyplot as plt

if '../' not in sys.path:
    sys.path.append('../')

In [6]:
env = gym.envs.make("Breakout-v0")

In [7]:
VALID_ACTIONS = [0, 1, 2, 3]


In [3]:
class StateProcessor():
    """
    Processes a raw Atari images. Resizes it and converts it to grayscale.
    """
    def __init__(self):
        # Build the Tensorflow graph
        self.input_state = tf.keras.Input(shape=[210, 160, 3], dtype=tf.uint8)
        self.output = tf.image.rgb_to_grayscale(self.input_state)
        self.output = tf.image.crop_to_bounding_box(self.output, 34, 0, 160, 160)
        self.output = tf.image.resize_images(self.output, [84, 84], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        self.output = tf.squeeze(self.output)

    def process(self, sess, state):
        """
        Args:
            sess: A Tensorflow session object
            state: A [210, 160, 3] Atari RGB State

        Returns:
            A processed [84, 84] state representing grayscale values.
        """
        return 

In [4]:
class Estimator(keras.layers.Layer):
    '''
    Q-value estimator neural network
    This network is used for both the Q-Network and the Target Network.
    '''

    def __init__(self, summaries_dir=None):
        self.summary_writer = None

    def build_model(self,):
        self.X_pl = tf.keras.Input(
            shape=[None, 84, 84, 4], dtype=tf.uint8, name='X')
        self.y_pl = tf.keras.Input(
            shape=[None], dtype=tf.float32, name='y')
        self.actions_pl = tf.keras.Input(
            shape=[None], dtype=tf.int32, name="actions")

        X = self.X_pl / 255.0
        batch_size = tf.shape(self.X_pl)[0]

        conv1 = tf.keras.layers.Conv2D(
            filters=32, kernel_size=8, strides=4, activation='relu')(X)
        conv2 = tf.keras.layers.Conv2D(
            filters=64, kernel_size=4, strides=2, activation='relu')(conv1)
        conv3 = tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=1, activation='relu')(conv2)

        flattened = tf.keras.layers.Flatten()(conv3)
        fc1 = tf.keras.layers.Dense(512)(flattened)
        self.predictions = tf.keras.Dense(len(VALID_ACTIONS))(fc1)
        
        